## About this notebook:

This notebook extracts information from single cells in the 4i experiment. To customize calculated parameters read documentation of scikit-image regionprops: https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.regionprops.

Input:
- data frames for selected wells
- labels
- selected round (anchor of alignment)
- punchmask (mask of excluded regions)
- aligned tiffs

Output:
- data frames with features of nuclei and rings

## Fill in info about the experiment to process

In [1]:
# pathway to a directory with data frames (ex. df)
path_df = r''

# pathway to segmented cells (ex. im_segmented)
path_labels = r''
label_round = 0 # 0-indexed

# pathway to punchmasks (ex. masks)
path_mask = r''

# pathway to aligned tiffs (ex. aligned_tiffs)
path_tiffs = r''

# pathway to save output data frames with info about cells (ex. cell data)
path_save = r''

well_list = ['A4','B4']

In [2]:
# settings for properties to calculate
properties = ['label', 'area','centroid','orientation','major_axis_length','minor_axis_length','bbox','mean_intensity']
properties_ring = ['label','mean_intensity']

## Prepare for processing

In [3]:
import os
import pickle5 as pickle
import numpy as np
import pandas as pd
from tifffile import imread
from skimage.measure import regionprops_table
from skimage.segmentation import clear_border
from skimage.draw import polygon
from ring_functions import make_rings

## Process

In [24]:
for selected_well in well_list:
    
    print(f'Processing well {selected_well}.')
    
    # open data frame
    df_name = f'df_{selected_well}.pkl'
    df = pd.read_pickle(os.path.join(path_df,df_name))
    
    ##############################################################
    
    # open labels
    print('Loading labels.')
    labels_path = os.path.join(path_labels,selected_well,f'labels_{selected_well}_round_{str(label_round).zfill(3)}.tif')
    labels = imread(labels_path)
    
    # clear border objects
    labels = clear_border(labels)
    
    ##############################################################
    
    # open punchmask
    print('Loading mask.')
    
    # translate shapes into an image
    mask = np.zeros(labels.shape).astype(int)
    
    try:
        mask_path = os.path.join(path_mask,f'mask_{selected_well}.pkl')
        sh = pickle.load( open( mask_path, "rb" ) )

        for reg in sh:
            rr, cc = polygon(reg[:,0],reg[:,1]) 
            mask[rr,cc] = 1
            
    except:
        print('Mask not found.')
        
    
    ##############################################################
    
    # open intensity images
    print('Loading intensity images.')
    int_im = []
    for ind,signal in df.iterrows():
        
        im_path = os.path.join(path_tiffs,selected_well,signal.aligned_file_name)
        im = imread(im_path)
        
        int_im.append(im)
        
    int_im.append(mask)    
        
    int_im = np.array(int_im)
    int_im = np.moveaxis(int_im,0,2)

        
    ##############################################################
        
    # calculate properties of nuclei
    print('Quantifying nuclei.')
    nuclei_df = pd.DataFrame(regionprops_table(labels, properties=properties,intensity_image=int_im))
        
    # adjust names
    channel_list = ['_'.join([str(int(x)).zfill(2),y]) for x,y in zip(df.nameRound,df.signal.to_list())]
    df['round_channel'] = channel_list
    
    intensity_names = [f'{x}_nuc_mean' for x in df.round_channel]
    intensity_names.append('nuc_mask')

    nuclei_df.columns=list(nuclei_df.columns[:-len(df)-1])+intensity_names
    
    ##############################################################
        
    # generate rings
    print('Generating cytoplasmic rings.')
    rings = make_rings(labels,width=6,gap=1)
    
    # calculate properties of rings
    print('Quantifying cytoplasmic rings.')
    rings_df = pd.DataFrame(regionprops_table(rings, properties=properties_ring,intensity_image=int_im))
    
    # adjust names
    intensity_names = [f'{x}_ring_mean' for x in df.round_channel]
    intensity_names.append('ring_mask')
    
    rings_df.columns=list(rings_df.columns[:-len(df)-1])+intensity_names

    ##############################################################
        
    # merging data frames
    cell_data = pd.merge(nuclei_df,rings_df,how='inner',on='label',suffixes=('_nuc', '_ring'))
    
    ##############################################################
        
    # add info and save 
    cell_data['well'] = selected_well
    
    cell_data.to_pickle(os.path.join(path_save,f'cell_data_{selected_well}_df.pkl'),protocol=4)
    

Processing well A4.
Loading labels.
Loading mask.
Loading intensity images.
Quantifying nuclei.
Generating cytoplasmic rings.
Quantifying cytoplasmic rings.
Processing well B4.
Loading labels.
Loading mask.
Mask not found.
Loading intensity images.
Quantifying nuclei.
Generating cytoplasmic rings.
Quantifying cytoplasmic rings.
